# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import userdata
import os
# Retrieve the API key from Colab secrets
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# Set it as an environment variable, which LangChain often uses by default
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
print("OpenAI API key loaded and set as environment variable.")

OpenAI API key loaded and set as environment variable.


In [4]:
!pip install pandas

In [13]:
import pandas as pd
df = pd.read_csv(r'/Data.csv')

In [14]:
df.head(7)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \r\n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


In [15]:
df.shape

(7, 2)

## LLMChain

In [10]:
!pip install langchain langchain-openai python-dotenv

INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.5 MB/s eta 0:00:00


In [ ]:
''' from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser '''

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain

In [16]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.5)


In [17]:
prompt = ChatPromptTemplate.from_template( "Create an attractive and compelling description for the product: {product}"


)

In [18]:
chain = LLMChain(llm=llm, prompt=prompt)

/tmp/ipython-input-2127810552.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [19]:
product = "Queen Size Sheet Set"
chain.run(product)
#chain.invoke({"product": product})


/tmp/ipython-input-1642471708.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


"Indulge in luxurious comfort with our Queen Size Sheet Set. Made from ultra-soft, high-quality materials, these sheets are designed to provide you with the perfect night's sleep. The elegant design and classic color options will effortlessly complement any bedroom decor. With deep pockets to fit mattresses up to 18 inches thick, our sheet set is not only stylish but also practical. Treat yourself to the ultimate in relaxation and upgrade your bedding with our Queen Size Sheet Set today."

## SimpleSequentialChain

In [20]:
from langchain.chains import SimpleSequentialChain

In [25]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
        model="gpt-3.5-turbo", temperature=0.9)
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "As an expert in product marketing, analyze the following product description:\n\n{product_description}\n\n"
    "Based on this, suggest 3 concise, catchy, and benefit-oriented taglines. "
    "Format them as a numbered list."
)
# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [26]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on these taglines for a product:\n\n{taglines_from_first_chain}\n\n"
    "Suggest a creative, short social media post (e.g., for Instagram or X) "
    "that incorporates one of these taglines. Include relevant emojis and hashtags."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [27]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [29]:

overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
1. "Experience luxurious comfort with our Queen Size Sheet Set"
2. "Upgrade your bedding with our premium Queen Size Sheet Set"
3. "Sleep like royalty on our Queen Size Sheet Set"
"Sleep like royalty every night with our luxurious Queen Size Sheet Set 👑✨ Upgrade your bedding and experience the ultimate comfort. #QueenSizeSheetSet #LuxuryBedding #SleepLikeRoyalty"

> Finished chain.


'"Sleep like royalty every night with our luxurious Queen Size Sheet Set 👑✨ Upgrade your bedding and experience the ultimate comfort. #QueenSizeSheetSet #LuxuryBedding #SleepLikeRoyalty"'

**Repeat the above twice for different products**

In [57]:
# Produto 1 (já usado)
product_1 = "Queen Size Sheet Set"
print("=== Produto 1 ===")
print(overall_simple_chain.run(product_1))

# Produto 2
product_2 = "Stainless Steel Water Bottle"
print("\n=== Produto 2 ===")
print(overall_simple_chain.run(product_2))

# Produto 3
product_3 = "Wireless Noise-Cancelling Headphones"
print("\n=== Produto 3 ===")
print(overall_simple_chain.run(product_3))


=== Produto 1 ===


> Entering new SimpleSequentialChain chain...
1. "Sleep like royalty every night with our luxurious Queen Size Sheet Set"
2. "Upgrade your bedroom with our premium Queen Size Sheet Set for ultimate comfort"
3. "Transform your bed into a cozy oasis with our soft and stylish Queen Size Sheet Set"
"👑💤 Sleep like royalty every night with our luxurious Queen Size Sheet Set. Transform your bed into a cozy oasis and upgrade your bedroom with ultimate comfort. #LuxuryBedding #QueenSizeSheets #CozyNights"

> Finished chain.
"👑💤 Sleep like royalty every night with our luxurious Queen Size Sheet Set. Transform your bed into a cozy oasis and upgrade your bedroom with ultimate comfort. #LuxuryBedding #QueenSizeSheets #CozyNights"

=== Produto 2 ===


> Entering new SimpleSequentialChain chain...
1. "Stay Hydrated in Style with our Stainless Steel Water Bottle"
2. "Sip with Confidence - Durable and Eco-Friendly Stainless Steel Water Bottle"
3. "Refresh on-the-go with our Sleek St

## SequentialChain

In [30]:
from langchain.chains import SequentialChain

In [31]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  """
    You will receive a product review in any language.
    Translate the review into English while keeping the original tone.
    Review: {input}
    """
)

chain_one = LLMChain(llm=llm, prompt=first_prompt,
                     output_key="english_review"
                    )

In [39]:
second_prompt = ChatPromptTemplate.from_template(
    """
    You will receive a product review.
    Write a clear 2–3 sentence summary highlighting the main points.
    Review: {english_review}
    """
)

chain_two = LLMChain(llm=llm, prompt=second_prompt,
                     output_key="summary"
                    )

In [40]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
     """
    Translate the following text into Spanish.
    Text: {summary}
    """
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translated_summary"
                      )

In [41]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
     """
    Use the translated text and the summary to write a short and polite
    follow-up message to the customer, thanking them for their review
    and offering further assistance.

    Translated text: {english_review}
    Summary: {translated_summary}
    """
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [42]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["input"],
    output_variables=["english_review", "summary", "translated_summary", "followup_message"],
    verbose=True
)

In [37]:
review = df.Review[5]
overall_chain(review)

/tmp/ipython-input-1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'input': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'english_review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in the store and the taste is much better... Old batch or counterfeit!?",
 'summary': "The reviewer finds the taste of the product mediocre and the foam doesn't hold up well. They suspect that it may be an old batch or a counterfeit product, as they have purchased the same item elsewhere with a much better taste.",
 'translated_summary': 'Encuentro el sabor mediocre. La espuma no se mantiene, es extraño. Compro los mismos en el comercio y el sabor es mucho mejor... ¿Lote viejo o falsificación!?',
 'followup_message': "Dear [Customer],\n\nThank you for sharing your feedback with us. We are sorry to hear that you found the taste of our product to be mediocre and that the foam didn't hold up well. We value your

**Repeat the above twice for different products or reviews**

In [59]:
# Segunda review
review = df.Review[2]
overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'input': "This mattress had a small hole in the top of it (took forever to find where it was), and the patches that they provide did not work, maybe because it's the top of the mattress where it's kind of like fabric and a patch won't stick. Maybe I got unlucky with a defective mattress, but where's quality assurance for this company? That flat out should not happen. Emphasis on flat. Cause that's what the mattress was. Seriously horrible experience, ruined my friend's stay with me. Then they make you ship it back instead of just providing a refund, which is also super annoying to pack up an air mattress and take it to the UPS store. This company is the worst, and this mattress is the worst.",
 'english_review': "Translation: This mattress had a small hole at the top (took a long time to find where it was), and the patches provided did not work, maybe because it's like fabric on the top of the mattress and a patch won't stick. Maybe I was unlucky with a defective mattress, but where i

In [60]:
review = df.Review[4]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'input': "\xa0I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can't be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It's not overuse or abuse. I'm very disappointed and will look for another. As I understand they will only replace once. Anyway, if you have one good luck.",
 'english_review': "Review: I really enjoyed this product. However, it seems to only last a few months. The company was fantastic in replacing the first one (the frother falls out of the handle and cannot be fixed). But unfortunately, after 4 months, my second one had the same issue. I only use the frother for coffee once a day, so it's not due to overuse or abuse. I am really let down by this and will now be searching for a different product. I believe they only replace it once. In any case, good luck if you decide to try it out.",
 'summary': 'The reviewer en

## Router Chain

In [45]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [46]:
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template
    },
    {
        "name": "computer science",
        "description": "Good for answering computer science questions",
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [47]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [48]:
llm = ChatOpenAI(temperature=0)

In [49]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [50]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [51]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [52]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [53]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

/tmp/ipython-input-3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [54]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body is characterized by a continuous spectrum of wavelengths and intensities that depend only on the temperature of the body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body is proportional to the frequency of the radiation and the temperature of the body. Black body radiation plays a key role in understanding the behavior of objects at high temperatures and in the development of quantum mechanics."

In [55]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [56]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells, maintaining the integrity and continuity of the organism's genetic code.\n\nAdditionally, DNA serves as a storage system for genetic information, allowing for the transmission of traits from one generation to the next through the process of inheritance. This is essential for the survival and evolution of species, as it allows for genetic variation and adaptation to changing environments over time."

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [61]:
chain.run("How does quantum entanglement violate classical locality?")



> Entering new MultiPromptChain chain...
physics: {'input': 'How does quantum entanglement violate classical locality?'}
> Finished chain.


'Quantum entanglement violates classical locality because it allows two particles to become connected in such a way that the state of one particle instantly affects the state of the other, regardless of the distance between them. This phenomenon is known as "spooky action at a distance" and goes against the principles of classical physics, which assume that information cannot travel faster than the speed of light. Quantum entanglement shows that particles can be interconnected in a way that transcends classical notions of space and time.'

In [62]:
chain.run("Tell me why the Roman Empire fell, but summarize like I'm 12 years old.")



> Entering new MultiPromptChain chain...
History: {'input': "Tell me why the Roman Empire fell, but summarize like I'm 12 years old."}
> Finished chain.


"The Roman Empire fell because it got too big and had trouble keeping everything together. There were a lot of problems like invasions from other groups, economic issues, and political corruption. Eventually, the empire split into smaller parts and couldn't stay strong enough to survive."